## Necessary Imports

In [200]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

## Define the Connection

In [201]:
conn_str = (
        r"Driver=ODBC Driver 17 for SQL Server;"
        r"Server=DS01;"
        r"Database=Sandbox_Intern_LD;"
        r"Trusted_Connection=yes;"
    )

conn_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_str})

engine = create_engine(conn_url)

## Read the Table

In [202]:
# query = 'SELECT * FROM LeanHogs_re20240617 ORDER BY MarketTradeDate DESC;'
query = 'SELECT * FROM LeanHogs_re20240617 AS t1 WHERE Volume = (SELECT Max(Volume) FROM LeanHogs_re20240617 AS t2 WHERE t1.MarketTradeDate = t2.MarketTradeDate) ORDER BY MarketTradeDate DESC;'
df = pd.read_sql_query(query, con=engine)
df.head()

,MarketTradeDate,FuturesContractSpecId,OpenPx,HighPx,LowPx,SettlementPx,Volume,OpenInterest,DailyOK,ContractDate,ExpiryDate,LastTradingUTCDateTime,FirstNoticeDate,SpotContractInd
0,2024-06-14,49,-1.000000e+09,-1.000000e+09,-1.000000e+09,77.925,0,-1000000000,True,2025-10-01,2025-10-14 17:00:00,2025-10-14 17:00:00,None,0
1,2024-06-14,49,7.972500e+01,8.027500e+01,7.870000e+01,80.200,0,-1000000000,True,2025-04-01,2025-04-14 17:00:00,2025-04-14 17:00:00,None,0
2,2024-06-14,49,9.190000e+01,9.255000e+01,9.190000e+01,92.550,0,-1000000000,True,2025-08-01,2025-08-14 17:00:00,2025-08-14 17:00:00,None,0
3,2024-06-14,49,7.575000e+01,7.697500e+01,7.415000e+01,76.750,0,-1000000000,True,2024-10-01,2024-10-14 17:00:00,2024-10-14 17:00:00,None,0
4,2024-06-14,49,7.500000e+01,7.532500e+01,7.355000e+01,75.225,0,-1000000000,True,2025-02-01,2025-02-14 18:00:00,2025-02-14 18:00:00,None,0


## Generate Timeseries

In [203]:
df['ContractDate'] = pd.to_datetime(df['ContractDate'], errors='coerce')
df['MarketTradeDate'] = pd.to_datetime(df['MarketTradeDate'], errors='coerce')
# df = df[df['ContractDate'].dt.month == 7]
df.head()

,MarketTradeDate,FuturesContractSpecId,OpenPx,HighPx,LowPx,SettlementPx,Volume,OpenInterest,DailyOK,ContractDate,ExpiryDate,LastTradingUTCDateTime,FirstNoticeDate,SpotContractInd
5,2024-06-14,49,92.350,94.325,90.400,93.650,0,-1000000000,True,2024-07-01,2024-07-15 17:00:00,2024-07-15 17:00:00,None,0
12,2024-06-14,49,91.850,93.350,91.850,93.200,0,-1000000000,True,2025-07-01,2025-07-15 17:00:00,2025-07-15 17:00:00,None,0
20,2024-06-13,49,93.175,93.475,93.100,93.375,102,512,True,2025-07-01,2025-07-15 17:00:00,2025-07-15 17:00:00,None,0
22,2024-06-13,49,92.975,94.575,92.475,92.775,32466,42351,True,2024-07-01,2024-07-15 17:00:00,2024-07-15 17:00:00,None,0
26,2024-06-12,49,93.700,94.000,93.650,93.825,51,456,True,2025-07-01,2025-07-15 17:00:00,2025-07-15 17:00:00,None,0


In [204]:
# for i, row in df.iterrows():
#     # print(row['ContractDate'].year, row['MarketTradeDate'].year)
#     if row['ContractDate'].year - 1 == row['MarketTradeDate'].year:
#         if row['MarketTradeDate'].month >= 7:
#             continue
#     if row['ContractDate'].year == row['MarketTradeDate'].year:
#         if row['MarketTradeDate'].month < 7:
#             continue
#     df.drop(i, inplace=True)

# df.head()

,MarketTradeDate,FuturesContractSpecId,OpenPx,HighPx,LowPx,SettlementPx,Volume,OpenInterest,DailyOK,ContractDate,ExpiryDate,LastTradingUTCDateTime,FirstNoticeDate,SpotContractInd
5,2024-06-14,49,92.350,94.325,90.400,93.650,0,-1000000000,True,2024-07-01,2024-07-15 17:00:00,2024-07-15 17:00:00,None,0
22,2024-06-13,49,92.975,94.575,92.475,92.775,32466,42351,True,2024-07-01,2024-07-15 17:00:00,2024-07-15 17:00:00,None,0
33,2024-06-12,49,92.925,94.750,92.525,93.700,37327,45562,True,2024-07-01,2024-07-15 17:00:00,2024-07-15 17:00:00,None,0
48,2024-06-11,49,93.250,93.800,92.075,92.275,38325,53310,True,2024-07-01,2024-07-15 17:00:00,2024-07-15 17:00:00,None,0
58,2024-06-10,49,93.500,94.050,92.475,92.900,33134,62074,True,2024-07-01,2024-07-15 17:00:00,2024-07-15 17:00:00,None,0


In [205]:
# for i, row in df.iterrows():
#     if i % 1000 == 0:
#         print(f"{i}/{len(df)}")
#     contract = row['ContractDate']
#     market = row['MarketTradeDate']
#     if contract.month == 1:
#         if market.month == 12 and market.year == contract.year - 1:
#             continue
#     if contract.month - 1 == market.month:
#         if contract.year == market.year:
#             continue
#     df.drop(i, inplace=True)

# df.head()

In [206]:
# df = df[df['SpotContractInd'] == 1]
# df.head()

## Restrict Columns and Save as CSV

In [207]:
df.reset_index(inplace=True)
df.drop(columns=['FuturesContractSpecId', 'DailyOK', 'ExpiryDate', 'LastTradingUTCDateTime', 'FirstNoticeDate', 'SpotContractInd', 'index'], inplace=True)
df.head()

,MarketTradeDate,OpenPx,HighPx,LowPx,SettlementPx,Volume,OpenInterest,ContractDate
0,2024-06-14,92.350,94.325,90.400,93.650,0,-1000000000,2024-07-01
1,2024-06-13,92.975,94.575,92.475,92.775,32466,42351,2024-07-01
2,2024-06-12,92.925,94.750,92.525,93.700,37327,45562,2024-07-01
3,2024-06-11,93.250,93.800,92.075,92.275,38325,53310,2024-07-01
4,2024-06-10,93.500,94.050,92.475,92.900,33134,62074,2024-07-01


In [208]:
df.to_csv("../Data/SQL_LHF.csv")

## 